In [2]:
! pip install -U prettytable

In [31]:
import pandas as pd
from prettytable import prettytable
import requests
import json

In [33]:
#api_key = pd.read_csv('data/bls_api.txt')

In [38]:

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ["CUUR0000SA0", "CES3000000006", "JTU300000000000000JOL", 
                                "JTU300000000000000HIL", "JTU300000000000000TSL"],
                   "startyear":"2001",
                   "endyear":"2021", 
                   "calculations":"true",
                   "registrationkey":"92eed6734bf5495f88972d9ba4106350"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [39]:
print(x)
# JTU300000000000000TSL = Manufacturing Separations 

+-----------------------+------+--------+-------+-----------+
|       series id       | year | period | value | footnotes |
+-----------------------+------+--------+-------+-----------+
| JTU300000000000000TSL | 2020 |  M12   |  344  |           |
| JTU300000000000000TSL | 2020 |  M11   |  328  |           |
| JTU300000000000000TSL | 2020 |  M10   |  373  |           |
| JTU300000000000000TSL | 2020 |  M09   |  393  |           |
| JTU300000000000000TSL | 2020 |  M08   |  430  |           |
| JTU300000000000000TSL | 2020 |  M07   |  395  |           |
| JTU300000000000000TSL | 2020 |  M06   |  426  |           |
| JTU300000000000000TSL | 2020 |  M05   |  319  |           |
| JTU300000000000000TSL | 2020 |  M04   |  859  |           |
| JTU300000000000000TSL | 2020 |  M03   |  880  |           |
| JTU300000000000000TSL | 2020 |  M02   |  301  |           |
| JTU300000000000000TSL | 2020 |  M01   |  343  |           |
| JTU300000000000000TSL | 2019 |  M12   |  293  |           |
| JTU300

In [13]:
# JTU300000000000000HIL = Manufacturing Hires 
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL', 
                                'JTU300000000000000HIL'],
                                "startyear":"2001",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MHires=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MHires.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MHires.get_string())
    output.close()

In [14]:
print(x_MHires)

+-----------------------+------+--------+-------+-----------+
|       series id       | year | period | value | footnotes |
+-----------------------+------+--------+-------+-----------+
| JTU300000000000000HIL | 2010 |  M12   |  176  |           |
| JTU300000000000000HIL | 2010 |  M11   |  238  |           |
| JTU300000000000000HIL | 2010 |  M10   |  271  |           |
| JTU300000000000000HIL | 2010 |  M09   |  265  |           |
| JTU300000000000000HIL | 2010 |  M08   |  292  |           |
| JTU300000000000000HIL | 2010 |  M07   |  309  |           |
| JTU300000000000000HIL | 2010 |  M06   |  308  |           |
| JTU300000000000000HIL | 2010 |  M05   |  291  |           |
| JTU300000000000000HIL | 2010 |  M04   |  291  |           |
| JTU300000000000000HIL | 2010 |  M03   |  250  |           |
| JTU300000000000000HIL | 2010 |  M02   |  225  |           |
| JTU300000000000000HIL | 2010 |  M01   |  240  |           |
| JTU300000000000000HIL | 2009 |  M12   |  148  |           |
| JTU300

In [15]:
#JTU300000000000000JOL = Manufacturing all areas Job Openings
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL' 
                                ],
                                "startyear":"2001",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MJobs=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MJobs.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MJobs.get_string())
    output.close()

In [ ]:
print(x_MJobs)

In [ ]:
#JTU300000000000000JOL = Manufacturing all areas Job Openings
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL' 
                                ],
                                "startyear":"2010",
                                "endyear":"2020" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MJobs2=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MJobs2.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MJobs2.get_string())
    output.close()

In [ ]:
print(x_MJobs2)

In [ ]:
#JTU300000000000000JOL = Manufacturing all areas Job Openings
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006', 'JTU300000000000000JOL' 
                                ],
                                "startyear":"2021",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MJobs3=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MJobs3.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MJobs3.get_string())
    output.close()

In [16]:
print(x_MJobs3)

+-----------------------+------+--------+-------+-----------+
|       series id       | year | period | value | footnotes |
+-----------------------+------+--------+-------+-----------+
| JTU300000000000000JOL | 2010 |  M12   |  171  |           |
| JTU300000000000000JOL | 2010 |  M11   |  207  |           |
| JTU300000000000000JOL | 2010 |  M10   |  198  |           |
| JTU300000000000000JOL | 2010 |  M09   |  189  |           |
| JTU300000000000000JOL | 2010 |  M08   |  188  |           |
| JTU300000000000000JOL | 2010 |  M07   |  241  |           |
| JTU300000000000000JOL | 2010 |  M06   |  210  |           |
| JTU300000000000000JOL | 2010 |  M05   |  190  |           |
| JTU300000000000000JOL | 2010 |  M04   |  191  |           |
| JTU300000000000000JOL | 2010 |  M03   |  153  |           |
| JTU300000000000000JOL | 2010 |  M02   |  157  |           |
| JTU300000000000000JOL | 2010 |  M01   |  142  |           |
| JTU300000000000000JOL | 2009 |  M12   |  152  |           |
| JTU300

In [23]:
#CES3000000006 = Manufacturing Employed
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006' 
                                ],
                                "startyear":"2001",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MWorkers=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MWorkers.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MWorkers.get_string())
    output.close()

In [24]:
print(x_MWorkers)

+---------------+------+--------+-------+-----------+
|   series id   | year | period | value | footnotes |
+---------------+------+--------+-------+-----------+
| CES3000000006 | 2010 |  M12   |  8117 |           |
| CES3000000006 | 2010 |  M11   |  8103 |           |
| CES3000000006 | 2010 |  M10   |  8096 |           |
| CES3000000006 | 2010 |  M09   |  8098 |           |
| CES3000000006 | 2010 |  M08   |  8099 |           |
| CES3000000006 | 2010 |  M07   |  8106 |           |
| CES3000000006 | 2010 |  M06   |  8100 |           |
| CES3000000006 | 2010 |  M05   |  8078 |           |
| CES3000000006 | 2010 |  M04   |  8050 |           |
| CES3000000006 | 2010 |  M03   |  8027 |           |
| CES3000000006 | 2010 |  M02   |  8032 |           |
| CES3000000006 | 2010 |  M01   |  8034 |           |
| CES3000000006 | 2009 |  M12   |  8050 |           |
| CES3000000006 | 2009 |  M11   |  8079 |           |
| CES3000000006 | 2009 |  M10   |  8100 |           |
| CES3000000006 | 2009 |  M0

In [25]:
#CES3000000006 = Manufacturing Employed
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006' 
                                ],
                                "startyear":"2011",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MWorkers2=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MWorkers2.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MWorkers2.get_string())
    output.close()

In [26]:
print(x_MWorkers2)

+---------------+------+--------+-------+-----------+
|   series id   | year | period | value | footnotes |
+---------------+------+--------+-------+-----------+
| CES3000000006 | 2020 |  M12   |  8471 |           |
| CES3000000006 | 2020 |  M11   |  8441 |           |
| CES3000000006 | 2020 |  M10   |  8406 |           |
| CES3000000006 | 2020 |  M09   |  8390 |           |
| CES3000000006 | 2020 |  M08   |  8364 |           |
| CES3000000006 | 2020 |  M07   |  8354 |           |
| CES3000000006 | 2020 |  M06   |  8326 |           |
| CES3000000006 | 2020 |  M05   |  7962 |           |
| CES3000000006 | 2020 |  M04   |  7776 |           |
| CES3000000006 | 2020 |  M03   |  8860 |           |
| CES3000000006 | 2020 |  M02   |  8932 |           |
| CES3000000006 | 2020 |  M01   |  8916 |           |
| CES3000000006 | 2019 |  M12   |  8941 |           |
| CES3000000006 | 2019 |  M11   |  8944 |           |
| CES3000000006 | 2019 |  M10   |  8900 |           |
| CES3000000006 | 2019 |  M0

In [28]:
#CES3000000006 = Manufacturing Employed
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'CES3000000006' 
                                ],
                                "startyear":"2021",
                                "endyear":"2021" 
                                #"calculations":"true"
                  })
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x_MWorkers3=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x_MWorkers3.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x_MWorkers3.get_string())
    output.close()

In [29]:
print(x_MWorkers3)

+---------------+------+--------+-------+-------------+
|   series id   | year | period | value |  footnotes  |
+---------------+------+--------+-------+-------------+
| CES3000000006 | 2021 |  M04   |  8467 | preliminary |
| CES3000000006 | 2021 |  M03   |  8514 | preliminary |
| CES3000000006 | 2021 |  M02   |  8484 |             |
| CES3000000006 | 2021 |  M01   |  8458 |             |
+---------------+------+--------+-------+-------------+
